# FiLM: Frame Interpolation for Large Motion

This notebook is a convenience wrapper around a pre-trained model released by ML researchers at Google. Learn more about their work here:

* https://github.com/google-research/frame-interpolation

---

Notebook authored by David Marx ([@DigThatData](https://twitter.com/DigThatData)), released under the MIT license.

If you experience any issue with or have suggestions regarding this notebook, please report them here:
* https://github.com/pytti-tools/pytti-notebook/issues/new


In [ ]:
# This top bit only neneds to be run once
# ...may as well set up the notebook to
# assume the user is only going to run all
# of setup once.
from pathlib import Path
import os

drive_mounted = False
gdrive_fpath = '.'
local_path = '/content/'

##############################

#@markdown # 1. Setup Workspace

#@markdown Run this cell to perform setup.

#@markdown Mounting your google drive is optional.
#@markdown If you mount your drive, code and models will be downloaded to it.
#@markdown This should reduce setup time after your first run.

###################

# Optionally Mount GDrive

mount_gdrive = False # @param{type:"boolean"}
if mount_gdrive and not drive_mounted:
    from google.colab import drive

    gdrive_mountpoint = '/content/drive/' #@param{type:"string"}
    gdrive_subdirectory = 'MyDrive/interpolation' #@param{type:"string"}
    gdrive_fpath = str(Path(gdrive_mountpoint) / gdrive_subdirectory)
    try:
        drive.mount(gdrive_mountpoint, force_remount = True)
        !mkdir -p {gdrive_fpath}
        %cd {gdrive_fpath}
        local_path = gdrive_fpath
        drive_mounted = True
    except OSError:
        print(
            "\n\n-----[PYTTI-TOOLS]-------\n\n"
            "If you received a scary OSError and your drive"
            " was already mounted, ignore it."
            "\n\n-----[PYTTI-TOOLS]-------\n\n"
            )
        raise

#####################

# Perform rest of setup

if not Path('./frame-interpolation').exists():
    !git clone https://github.com/pytti-tools/frame-interpolation

try:
    import frame_interpolation
except ModuleNotFoundError:
    %pip install -r ./frame-interpolation/requirements_colab.txt
    %pip install ./frame-interpolation

#url = "https://drive.google.com/drive/folders/1GhVNBPq20X7eaMsesydQ774CgGcDGkc6?usp=sharing"
share_id = "1GhVNBPq20X7eaMsesydQ774CgGcDGkc6"

if not (Path(local_path) / 'saved_model').exists():
    !pip install --upgrade gdown
    !gdown --folder {share_id}

# create default frame
!mkdir -p frames

In [ ]:
# @title 1.5:  Convert the video into frames
import cv2
import os

def video_to_frames(video_path, output_dir):
  """Converts a video into frames and saves them to an output directory.

  Args:
    video_path: Path to the video file.
    output_dir: Path to the output directory where frames will be saved.
  """
  # Create the output directory if it doesn't exist
  os.makedirs(output_dir, exist_ok=True)

  # Open the video file
  vidcap = cv2.VideoCapture(video_path)

  # Get the video's frame rate
  fps = vidcap.get(cv2.CAP_PROP_FPS)

  # Initialize frame counter
  count = 0

  # Read frames from the video until it's finished
  success, image = vidcap.read()
  while success:
    # Save the current frame as an image file
    cv2.imwrite(os.path.join(output_dir, f"frame_{count:04d}.jpg"), image)

    # Increment the frame counter
    count += 1

    # Read the next frame
    success, image = vidcap.read()

  # Release the video capture object
  vidcap.release()

  print(f"Video converted to frames. {count} frames saved to {output_dir}")

# Example usage:
video_path = "path/to/your/video.mp4"  # Replace with your video file path
output_dir = "frames"  # Replace with your desired output directory

video_to_frames(video_path, output_dir)

In [ ]:
#@markdown # 2. Setup Interpolation

#@markdown Specify the directory containing your video frames with the `frames_dir` parameter.

frames_dir = "frames" #@param{'type':'string'}

#@markdown A single pass of the interpolation procedure adds a frame between each contiguous pair of frames in `frames_dir`.

#@markdown If you start with $n$ frames in `frames_dir` and set `recursive_interpolation_passes` to $k$, your total number of frames
#@markdown after interpolation will be:
#@markdown $$2^k (n-1) -1$$

recursive_interpolation_passes = 1 #@param{'type':'integer'}

#@markdown Check this box to generate a video output. If no output video will be generated, the FPS option can be ignored.
output_video = True #@param{'type':'boolean'}
output_video_fps = 30 #@param{'type':'number'}

In [ ]:
#@markdown # 3. Interpolate!
#@markdown Results will be saved in a subdirectory of `frames_dir` named "interpolated_frames"

from loguru import logger

logger.info("Beginning interpolation...")

if output_video:
  !python -m frame_interpolation.eval.interpolator_cli \
      --model_path ./saved_model \
      --pattern {frames_dir} \
      --fps {output_video_fps} \
      --output_video


else:
    !python -m frame_interpolation.eval.interpolator_cli \
      --model_path ./saved_model \
      --pattern {frames_dir} \

logger.info("Interpolation comlpete.")

```
MIT License

Copyright (c) 2022 David Marx

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.
```